In [17]:
import sys;print(sys.version)

3.10.8 (v3.10.8:aaaf517424, Oct 11 2022, 10:14:40) [Clang 13.0.0 (clang-1300.0.29.30)]


In [32]:
os.getcwd()

'/Users/biancaziliotto/Desktop/GitRepo/MLx2/src'

In [33]:
from MLP import MLP 

In [34]:
mlp = MLP(0, [], 3, 2)

IndexError: list index out of range